In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/passworddataset/password_data.sqlite


In [2]:
import sqlite3 

In [3]:
#sqlite connection
con = sqlite3.connect(r"/kaggle/input/passworddataset/password_data.sqlite")


In [4]:
data = pd.read_sql_query("SELECT * from Users",con)

In [5]:
data.shape

(100000, 3)

In [6]:
data.head()

,index,password,strength
0,0,zxe870819,1
1,1,xw46454nr23l,1
2,2,soporte13,1
3,3,accounts6000webhost.com,2
4,4,c443balg,1


In [7]:
#check for columns andremove unnecessary columns
data.columns

Index(['index', 'password', 'strength'], dtype='object')

In [8]:
#we can drop 'index'
data.drop(['index'], axis=1, inplace = True)

In [9]:
data.head()

,password,strength
0,zxe870819,1
1,xw46454nr23l,1
2,soporte13,1
3,accounts6000webhost.com,2
4,c443balg,1


In [10]:
#check for duplicates
data.duplicated().sum()   #no duplicate

0

In [11]:
#check for null
data.isnull().any().sum()  #no null

0

In [12]:
#datatype
data.dtypes

password    object
strength     int64
dtype: object

In [13]:
data['strength'].unique()

array([1, 2, 0])

# Semantic Analysis

In [14]:
type(data['password'][0])

str

In [15]:
#Passwords holds only numeric charracters??
data[data['password'].str.isnumeric()].shape

(26, 2)

In [16]:
data[data['password'].str.isalpha()].shape #alpha characters password

(50, 2)

In [17]:
#password having alpha numeric charcters
data[data['password'].str.isalnum()].shape 

(97203, 2)

In [18]:
#password having uppercase character??
print(data[data['password'].str.isupper()].shape)

(1506, 2)


In [19]:
#passwords having some speical chaparcter
import string

In [20]:
string.punctuation  #to get the special charcters

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
def findSpecialCharacter(row):
    """define a function
    if any row has any of speical character
    then reutrn 1 else 0
    Input: Row """
    for char in row:
        if char in string.punctuation:
            return 1
        else:
            pass

In [22]:
data[data['password'].apply(findSpecialCharacter)==1].shape

(2663, 2)

Therefore the summary is: 
1. Only Numeric password: 26
2. Only Alpha passowrd: 50
3. Alpha Numeric: 97203
4. Having only uppercase: 1506
5. Having Special Characters: 2663

# Feature Engineeringn

- find the frequency of lowercase, uppercase, digit, frequency in passwords

In [23]:
def freqLower(row):
    return len([char for char in row if char.islower()])/len(row) #dividing with row to normalise it

In [24]:
def freqUpper(row):
    return len([char for char in row if char.isupper()])/len(row) #dividing with row to normalise it

In [25]:
def freqDigit(row):
    return len([char for char in row if char.isdigit()])/len(row) #dividing with row to normalise it

In [26]:
np.round(data['password'].apply(freqLower), 3)

0        0.333
1        0.417
2        0.778
3        0.783
4        0.625
         ...  
99995    0.700
99996    0.800
99997    0.778
99998    0.667
99999    0.727
Name: password, Length: 100000, dtype: float64

In [27]:
data['lowercaseFreq'] = np.round(data['password'].apply(freqLower), 3)
data['uppercaseFreq'] = np.round(data['password'].apply(freqUpper), 3)
data['digiFreq'] =np.round(data['password'].apply(freqDigit), 3)

In [28]:
data.head()

,password,strength,lowercaseFreq,uppercaseFreq,digiFreq
0,zxe870819,1,0.333,0.0,0.667
1,xw46454nr23l,1,0.417,0.0,0.583
2,soporte13,1,0.778,0.0,0.222
3,accounts6000webhost.com,2,0.783,0.0,0.174
4,c443balg,1,0.625,0.0,0.375


In [29]:
# for special charcter
def freqSpeicalChar(row):
    specialChar = []
    for char in row:
        if not char.isalpha() and not char.isdigit():
            specialChar.append(char)
    return len(specialChar)/len(row)

In [30]:
data['specialCharFreq'] = np.round(data['password'].apply(freqSpeicalChar),3)

In [31]:
data.head()

,password,strength,lowercaseFreq,uppercaseFreq,digiFreq,specialCharFreq
0,zxe870819,1,0.333,0.0,0.667,0.000
1,xw46454nr23l,1,0.417,0.0,0.583,0.000
2,soporte13,1,0.778,0.0,0.222,0.000
3,accounts6000webhost.com,2,0.783,0.0,0.174,0.043
4,c443balg,1,0.625,0.0,0.375,0.000
